# Работа с парсером JSON

В этом разделе показано как использовать библиотеку промптов и JSON-парсер для генерации синонимов.

Для начала обновите библиотеку и импортируйте необходимые компоненты:

In [ ]:
pip install -qU gigachain

In [ ]:
from langchain.chains import LLMChain
from langchain_community.chat_models.gigachat import GigaChat
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.prompts import load_prompt
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate

При инициализации объекта GigaChat передайте ему авторизационные данные, которые будут использоваться для обращения к модели:

In [9]:
giga = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)

## Загрузка промпта

Для генерации используется промт `synonyms_generation.yaml`, доступный в [каталоге промптов](../../../../../hub/prompts):

```
Сгенерируй от {dataset_size_min} до {dataset_size_max} синонимов для слова "{subject}".
Результат верни в формате JSON списка без каких либо пояснений, например, ["синоним1", "синоним2", "синоним3", "синоним4"].
Не дублируй фразы.
```

Каталог содержит проверенные промпты, которые помогают получить более стабильный результат.

Загрузите промпт из GitHub-репозитория:

In [11]:
propmpt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(
            prompt=load_prompt("lc://prompts/synonyms/synonyms_generation.yaml")
        )
    ]
)

Вы также можете загружать промпты из локальных файлов:
    
```python
propmpt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(
            prompt = load_prompt("../../../../../hub/prompts/synonyms/synonyms_generation.yaml")
        )
    ]
)
```

## Сборка цепочки

Промпт указывает модели вернуть результат в виде JSON.
Поэтому, для получения структурированного результата используется JSON-парсер.

In [12]:
chain = LLMChain(llm=giga, prompt=propmpt, output_parser=SimpleJsonOutputParser())

Ниже представлены несколько примеров генерации.

In [14]:
chain.run(dataset_size_min=5, dataset_size_max=10, subject="транспортное средство")

['автомобиль', 'машина', 'автобус', 'грузовик', 'мотоцикл']

In [6]:
chain.run(dataset_size_min=5, dataset_size_max=10, subject="смартфон")

['мобильный телефон',
 'сотовый телефон',
 'карманный компьютер',
 'коммуникатор',
 'цифровой телефон']

In [7]:
chain.run(dataset_size_min=5, dataset_size_max=10, subject="молоток")

['молоток',
 'кувалда',
 'клепальник',
 'клепальный молоток',
 'клепальница',
 'клепальщик']